Reminder of Task 3.4 objectives:

This task will develop AI-based analytics tools to classify applications upon their IO behaviour in order to 
estimate their reproducibility. Once a family of reproducible applications is identified, its average behaviour 
is evaluated and will be used to perform: 
- optimizations on Ephemeral IO services resource allocation 
- optimizations on data placement 
- optimizations for energy efficiency 



Objectives are for M12-M36, but we need to provide inputs for:
- Task 3.1 – Instrumentation and Monitoring Design
- Task 3.2 – Application IO Instrumentation
- Task 3.3 – Infrastructure Monitoring and health check

Data Placement Problem belongs to a familiy of problems (similar to logistics, supply chain, asset allocation, scheduling) that can be solved using:
1. OR (Operational Research) heuristics [https://arxiv.org/pdf/2008.06319.pdf]
2. RL methods
    

Let's compare some results for known/similar problems when solved using RL, MILP (mixed-integer linear programming) and heuristic (best state of the art algorithmic solutions).
Keep in mind that problem variations can not always be tackled by MILP and heuristics, but it is a good way to start with.

<p float="center">
    <img src="knapsack_results.png" width="1200"/>
    <img src="asset_allocation_results.png" width="800"/>
</p>

In [16]:
!export https_proxy=http://129.183.4.13:8080
!export http_proxy=http://129.183.4.13:8080
!pip install pulp

Facility location problems are often solved as integer programs but are subject to many variations. 

In this context, facility location problems are often posed as follows: suppose there are $n$ facilities and $m$ customers. We wish to choose (1) which of the $n$ facilities to open, and (2) which (open) facilities to use to supply the $m$ customers, in order to satisfy some fixed demand at minimum cost. We introduce the following notation: let $f_{i}$ denote the (fixed) cost of opening facility $i$, $i=1,\dots ,n$. 

Let $c_{ij}$ denote the cost to ship a product from facility $i$ to customer $j$ for $i=1,\dots ,n $ and $j=1,\dots ,m$. 

Let $d_{j}$ denote the demand of customer $j$ for $j=1,\dots ,m$. 

Further suppose that each facility has a maximum output. Let $u_{i}$ denote the maximum amount of product that can be produced by facility $i$, that is, let $u_{i}$ denote the capacity of facility $i$.

\begin{array}{rl}
\min & \displaystyle\sum_{i=1}^n\sum_{j=1}^mc_{ij} d_j y_{ij}+\sum_{i=1}^nf_ix_i \\
\text{s.t.} & \displaystyle\sum_{i=1}^ny_{ij}=1 \text{ for all }j=1,\dots,m \\
& \displaystyle \sum_{j=1}^md_jy_{ij}\leqslant u_ix_i\text{ for all }i=1\dots,n \\
&y_{ij}\geqslant0\text{ for all }i=1,\dots,n \text{ and }j=1,\dots,m\\
&x_i\in\{0,1\}\text{ for all } i=1,\dots,n
\end{array}

<p float="center">
    <img src="https://scipbook.readthedocs.io/en/latest/_images/flp.png" width="500"/>
    <img src="data_placement_problem.png" width="500"/>
</p>

In [17]:
from pulp import *

#SETS
nodes = [1, 2, 3, 4, 5]  # customers are nodes, they express demand
disks = ['T1', 'T2', 'T3'] # disks are facility, they offer service with limited capacity

In [18]:
# Volumes requested by each node in GB  
# Could detail files/objects to be read/written
demand = {1:80, 2: 270, 3:250, 4: 160, 5: 180} #GB

# For instance (with lifecycle):
#demand = {1: ['obj8', 'obj13'],
#         2: ['obj12', 'obj13'],...}
# For instance:
#demand = {1: [60, 20],   # node 1 will request to read 60 GB read and to write 20GB of data 
#         2: [170, 100], ...} # # node 2 will request to read 60 GB read and to write 20GB of data 

In [19]:
# Here we convert initial cost of opening a facility to access latency for a specific Tier
# this duration as expressed here run once (as it was an assignment cost, cost of plugging a Tier to a Node)
# it should be affected at each access if needed
# it should be useful later for modeling seek/access, for hdd/tape may be statistical distrib
latency = {'T1': 10e-3, 'T2': 100e-3, 'T3': 1000e-3} # latencies in s  

In [20]:
# Here we give capacity for each Tier
# How to update this ?
capacity = {"T1": 300, "T2": 500, "T3": 1e3} # in GB

In [21]:
# Instead of expressing directly a cost function, we would like to minimize IO durations
# So we will use bandwidth as Tier properties in GB/s
bandwidth = {"T1": 10, "T2": 2, "T3": 1} # in GB/s 
# Here bandwidth is a sort of constant hardware spec
# It get not diminished when simultaneous transfer
# Is considered constant while it could vary with access mode R/W Seq/Stride/Rand req_size 
# We could imagine later minimizing energy consumption for instance
# But we'll need to be able to predict energy_consumption(file/object, node, Tier)
# bandwidth = {"T1": t1_energy_consumption, "T2": t2_energy_consumption, "T3": t3_energy_consumption}

In [22]:
# So we define a linear program problem (objective function is linear on variables)
# We choose to minimize total IO time for placing workflow data
prob = LpProblem("DataPlacementTimeMin", LpMinimize)

In [23]:
# First varuable assignement y: 
# y_i_j or y(i, j) 
# y_T1_node1 = 1 means that you place 100% of data requested by node 1 in T1.
y = LpVariable.dicts("Placement",
                         [(i, j) for i in disks for j in nodes], lowBound=0, cat='Binary') #lower band, >=0 #y(i, j)
# please notice that could be switched  later to cat='Continuous'

- [ ] switch to continuous for y

In [24]:
# We introduce here a second variable (legacy from CLF problem)
# x_T1 = 1 means that T1 will be used
# it is useful for assignment cost of assigning T1 to a job/workflow
x = LpVariable.dicts("DiskUse", disks, 0, 1, LpBinary) #x(i) indicates if disk i is accessed or not
# during this session/workflow
# it should be replaced by the number of times a disk is accessed (to infer latency)
# it could be more complex: x_T1 = list timestamps at wich T1 is accessed
# can allow to schedule access to T1 to avoid contention and manage finite bandwidth consumption
# it could force us to shift to ML/RL solution
# We'll we have to manage such dataflow scheduling?

In [25]:
# Here we introduce the objective function we want to minimize
# first member is the one-time latencies
# second member: demand[j]*y[(i, j)] = total volume to be placed in Tier T_i (over all nodes j) divided by the max bandwidth of T_i.
# second member = transfer time of data
prob += lpSum(latency[i]*x[i] for i in disks) + lpSum(demand[j]*y[(i, j)]/bandwidth[i] for i in disks for j in nodes) 

#prob += lpSum(demand[j]*y[(i, j)]/bandwidth[i] for i in disks for j in nodes) 
        # latency cost when using disk i       + transfer time 


- [ ] remove latencies

In [26]:
# CONSTRAINTS
#
for j in nodes:
    # data served on disks equals 100% from node j, no duplicates!
    prob += lpSum(y[(i, j)] for i in disks) == 1

- [ ] how to allow data duplication ?
it is relevant if the bandwidth is limited i think.

In [27]:
for i in disks:
    # data put on disk i from all nodes should be lower than its capacity
    prob += lpSum(y[(i, j)]*demand[j] for j in nodes) <= capacity[i]*x[i]
    
# if duplication is allowed, y >= 1.0 and this constraint holds.

In [28]:
# this constraint is a legacy for CLF problem
for j in nodes:
    for i in disks:
        # fraction of data served from node j to disk i is lower that demand of node j if disk i is activated
        prob += y[(i, j)] <= demand[j]*x[i] 
# well 0 <= y <= 1 and always <= to demand
# should be kept with y = volume not portion
# here we should remove demand
# finally the demand factor should be removed, this constraint states that y is 0 if disk is not activated.

In [29]:
#SOLVE
prob.solve()
print("Status:", LpStatus[prob.status])

Status: Optimal


In [30]:
# PRINT DECISION VARIABLES
print('--- Params ---')
print(f"nodes = {nodes}")
print(f"demand = {demand}")
print(f"disks capacity = {capacity}")
print(f"bandwidths = {bandwidth}")
print('\n--- Solution ---')
TOL = 1e-6
for i in disks:
    if x[i].value() > TOL:
        print("Establish connection with disk ", i)

for v in prob.variables():
    print(f"{v.name} = {v.varValue}")
    
# OPTIMAL SOLUTION
print(f"The cost of this placement is  = {value(prob.objective)} seconds")

--- Params ---
nodes = [1, 2, 3, 4, 5]
demand = {1: 80, 2: 270, 3: 250, 4: 160, 5: 180}
disks capacity = {'T1': 300, 'T2': 500, 'T3': 1000.0}
bandwidths = {'T1': 10, 'T2': 2, 'T3': 1}

--- Solution ---
Establish connection with disk  T1
Establish connection with disk  T2
Establish connection with disk  T3
DiskUse_T1 = 1.0
DiskUse_T2 = 1.0
DiskUse_T3 = 1.0
Placement_('T1',_1) = 0.0
Placement_('T1',_2) = 1.0
Placement_('T1',_3) = 0.0
Placement_('T1',_4) = 0.0
Placement_('T1',_5) = 0.0
Placement_('T2',_1) = 1.0
Placement_('T2',_2) = 0.0
Placement_('T2',_3) = 1.0
Placement_('T2',_4) = 1.0
Placement_('T2',_5) = 0.0
Placement_('T3',_1) = 0.0
Placement_('T3',_2) = 0.0
Placement_('T3',_3) = 0.0
Placement_('T3',_4) = 0.0
Placement_('T3',_5) = 1.0
The cost of this placement is  = 453.11 seconds


Outcomes:
- Which system to optimize on ?
    - known workflows ?
        some workflows could have higher priority (#1)
        we have information only from known workflows
        predicting wf behaviour is difficult
    - whole cluster ? 
        specific optimization imply relaxation on global optimization
        some workflows may "fail"
- How to get/feed/update parameters for data placement ? from  instrumentation/health checking ?
    - demand (node j)
    - bandwidths (disk i, node j)
    - lantencies (or IOPS) (disk i, node j)
    - energy_consumption functions for (disk i, node j)
    - health_check (disk i, node j)
    
- (specific to task 3.4) gym-like environment to benchmark data placement algorithms :
    - or-gym : didn't find facility location problem
    - Iroko: The Data Center RL Gym
    - ecole : wide range for combinatorial optimization problems (gym-compatible)